<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/50_ode/35_Runge_Kutta_Higher_Order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl



# 룽게-쿠타법 (RK4) : 고차 상미방<br>Runge-Kutta Method (RK4) : Higher Order ODE



## 단진자<br>Simple Pendulum



다음 미분 방정식은 단진자의 운동을 묘사한다.<br>
Following differential equation describes the motion of a simple pendulum.<br>
Ref : Wikipedia contributors, 'Pendulum (mathematics)', Wikipedia, The Free Encyclopedia, 2 June 2018, 13:28 UTC, <https://en.wikipedia.org/w/index.php?title=Pendulum_(mathematics)&oldid=844080803> [accessed 5 August 2018]



$$
\frac{d^2\theta}{dt^2} + \frac{g}{l}sin\theta = 0
$$



상태변수는 다음과 같다고 가정하자.<br>
Let's assume that the state variables are as follows.



$$
\mathbf{x}
=
\begin{pmatrix}
x_0\\
x_1
\end{pmatrix}
=
\begin{pmatrix}
\theta\\
\frac{d}{dt}\theta
\end{pmatrix}
$$



상태변수의 미분은 다음과 같다.<br>Differentiation of the state variables are as follows.



$$
\frac{d}{dt}
\begin{pmatrix}
    x_0\\
    x_1
\end{pmatrix} 
=
\begin{pmatrix}
    x_1\\
    -\frac{g}{l}sinx_0
\end{pmatrix} 
$$



python 함수로 구현해 보면 다음과 같을 것이다.<br>
One possible python implementation would be as follows.



In [ ]:
g_mpsps = 9.8
l_m = 0.3

legends = ('$\\theta(deg)$', '$\\frac{d}{dt}\\theta(deg/s)$')
ylabel = ''

# Initial state
x_0 = np.array([np.deg2rad(90), 0])

def pendulum_NL(t, x):
    """
    Parameters
    ==========
    x: array of theta and d(theta)/dt
    t: time value

    Return Value
    ============
    One dimensional array of dx/dt
    """

    return np.array([x[1], (-g_mpsps/l_m)*np.sin(x[0])])



룽게-쿠타법을 오일러법, 훈법과 비교해보자.<br>Let's compare the Runge-Kutta method with Euler method, and Heun's method.



In [ ]:
import ode_solver
import scipy.integrate as si



In [ ]:
py.figure(figsize=(12, 12))


# Time array
delta_t = 0.001

t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# *** scipy ***
sol_pendulum = si.solve_ivp(pendulum_NL, (t_sec_array[0], t_sec_array[-1]), x_0, t_eval=t_sec_array)
py.plot(sol_pendulum.t, sol_pendulum.y.T, 'o', label='solve_ivp', alpha=0.5)

# *** Euler ***
t_euler_out, x_euler_out = ode_solver.euler(pendulum_NL, t_sec_array, x_0)
py.plot(t_euler_out, x_euler_out, '-', label='Euler', alpha=0.5)

# *** Heun ***
t_heun__out, x_heun__out = ode_solver.heun(pendulum_NL, t_sec_array, x_0)
py.plot(t_heun__out, x_heun__out, '-', label='Heun', alpha=0.5)

# *** RK4 ***
t_rk4___out, x_rk4___out = ode_solver.rk4(pendulum_NL, t_sec_array, x_0)
py.plot(t_rk4___out, x_rk4___out, '.', label='RK4', alpha=0.5)


py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)



## 4차 선형 상미방<br>Fourth Order Linear ODE



아래 4차 선형 상미분 방정식에 대해서도 여러 해법을 적용해 보자.<br>
Let's apply various solvers to the following fourth order linear ODE.



$$
         \frac{d^4x}{dt^4} 
         + 12 \frac{d^3x}{dt^3} 
         + 54 \frac{d^2x}{dt^2} 
         + 108 \frac{dx}{dt} 
         + 80 x = 0
$$



$$
\mathbf{q} = \begin{pmatrix}q_0 & q_1 & q_2 & q_3 \end{pmatrix}^T = \begin{pmatrix}x & \frac{dx}{dt} & \frac{d^2x}{dt^2} & \frac{d^3x}{dt^3} \end{pmatrix}^T
$$



$$
\frac{d\mathbf{q}}{dt}
=
\begin{bmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
- 80 & - 108 & - 54 & -12
\end{bmatrix}
\begin{pmatrix}
q_0 \\ q_1 \\ q_2 \\ q_3
\end{pmatrix}
=
\mathbf{Aq}
$$



아래는 해당 4차 선형 상미방을 `dataclass`로 구현한 예이다. (Python 3.7 이후)<br>
Following is a `dataclass` implementation of the fourth order LODE. (Python 3.7 or later)



In [ ]:
import dataclasses


@dataclasses.dataclass
class LODE(object):
    A : np.ndarray
    def slope(self, t, q):
        """
        Parameters
        ==========
        q: array of q_0, q_1, q_2, ..., q_n
        t: time value

        Return Value
        ============
        One dimensional array of dq/dt
        """

        q_column = np.array(q).T
        qdot_column = self.A @ q_column

        qdot_array = np.array(qdot_column.T).flatten()

        return qdot_array


lode4 = LODE(np.array([
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [-80, -108, -54, -12],
]))

ylabel = '$\mathbf{q}$'

# Initial state
x_0 = np.array([1, 0, 0, 0])



시간 배열을 정한다.<br>
Define an array for time.



In [ ]:
delta_t = 0.08
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)



각 해법을 적용한다.<br>
Apply each solver.



In [ ]:
# *** scipy ***
sol = si.solve_ivp(lode4.slope, (t_sec_array[0], t_sec_array[-1]), x_0, t_eval=t_sec_array)

# *** Euler ***
t_euler_out, x_euler_out_ode4 = ode_solver.euler(lode4.slope, t_sec_array, x_0)

# *** Heun ***
t_heun__out, x_heun__out_ode4 = ode_solver.heun(lode4.slope, t_sec_array, x_0)

# *** RK4 ***
t_rk4___out, x_rk4___out_ode4 = ode_solver.rk4(lode4.slope, t_sec_array, x_0)



$q_i$ 별로 그리기 위해 넘파이 배열로 변환한다.<br>
To plot each $q_i$, convert to NumPy arrays.



In [ ]:
x_euler_out_ode4_array = np.array(x_euler_out_ode4)
x_heun__out_ode4_array = np.array(x_heun__out_ode4)
x_rk4___out_ode4_array = np.array(x_rk4___out_ode4)



In [ ]:
py.figure(figsize=(10, 10))

for i_state in range(x_euler_out_ode4_array.shape[1]):

    py.subplot(x_euler_out_ode4_array.shape[1], 1, i_state+1)
    py.plot(sol.t, sol.y[i_state, :], 'o', label='solve_ivp')
    py.plot(t_euler_out, x_euler_out_ode4_array[:, i_state], '-', label='Euler')
    py.plot(t_heun__out, x_heun__out_ode4_array[:, i_state], '-', label='Heun')
    py.plot(t_rk4___out, x_rk4___out_ode4_array[:, i_state], '.', label='RK4')
    py.xlabel('t(sec)')
    py.ylabel(f'$q_{i_state}$')
    py.legend(loc=1)
    py.grid(True)



## 연습 문제<br>Exercises



도전 과제 1: 다음 2계 선형 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Try This 1: Find the numerical solutions of the following second order linear ordinary equation using RK4 Method:

$$
\begin{align}
\frac{d^2}{dt^2}x(t) + 2\frac{d}{dt}x(t) + x(t) &= 0 \\
x(0) &= 0 \\
\frac{d}{dt}x(0) &= 1
\end{align}
$$



도전 과제 2: 다음 2계 선형 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Try This 2: Find the numerical solutions of the following second order linear ordinary equation using RK4 Method:

$$
\begin{align}
\frac{d^2}{dt^2}x(t) + \frac{d}{dt}x(t) + 4x(t) &= sin(t[rad]) \\
x(0) &= 0 \\
\frac{d}{dt}x(0) &= 0
\end{align}
$$



도전 과제 3: 다음 연립 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Try This 3: Find the numerical solutions of the following system of ordinary equations using RK4 Method:

$$
\begin{cases}
    \begin{align}
        \frac{d}{dt}x(t) &= 2 x(t) - y(t) \\
        \frac{d}{dt}y(t) &= 2.1 y(t) - x(t)
    \end{align}
\end{cases}\\
x(0) = y(0) = 1.0 \\
0 \le t \le 2
$$


ref : Zill & Cullen



도전 과제 4: 다음 연립 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Try This 4: Find the numerical solutions of the following system of ordinary equations using RK4 Method:

$$
\begin{cases}
    \begin{align}
        \frac{d}{dt}x(t) + \frac{d}{dt}y(t) + 2y(t)&= 0 \\
        \frac{d}{dt}x(t) - 3x(t) - 2 y(t) &= 0
    \end{align}
\end{cases}\\
x(0) = y(0) = 1.0 \\
0 \le t \le 2
$$


ref : Zill & Cullen



## 부록: 단진자의 방향장<br>Appendix: The direction field of a simple pendulum



각 상태에서 상태변수의 변화의 방향 $\left(\frac{d}{dt}\theta, \frac{d^2}{dt^2}\theta \right)$ 을 표시해 보자.<br>At each state, let's present the direction of state variable change $\left(\frac{d}{dt}\theta, \frac{d^2}{dt^2}\theta \right)$.



In [ ]:
import ode_plot



In [ ]:
time_list = []

# list of theta
theta_deg_array = np.arange(-540, 540+1, 30)
theta_rad_list = np.deg2rad(theta_deg_array)

# list of theta_dot
theta_dot_deg_array = np.arange(-540, 540+1, 45)
theta_dot_rad_list = np.deg2rad(theta_dot_deg_array)

# plot the direction filed
ode_plot.ode_slopes_2states_cartesian(pendulum_NL, theta_rad_list, theta_dot_rad_list, time_list)

# Convert pendulum solution curves to NumPy arrays
x_euler_out_array, x_heun__out_array, x_rk4___out_array = (
    np.array(x_euler_out), np.array(x_heun__out), np.array(x_rk4___out))

# Plot the solution curves
py.plot(
  py.rad2deg(x_euler_out_array[:, 0]),
  py.rad2deg(x_euler_out_array[:, 1]),
  '-', label='Euler', alpha=0.7
)
py.plot(
  py.rad2deg(x_heun__out_array[:, 0]),
  py.rad2deg(x_heun__out_array[:, 1]),
  '.-', label='Heun', alpha=0.7
)
py.plot(
  py.rad2deg(x_rk4___out_array[:, 0]),
  py.rad2deg(x_rk4___out_array[:, 1]),
  '-', label='RK4', alpha=0.7
)

ode_plot.set_axis(
    py.gca(), 
    theta_deg_array[0], theta_deg_array[-1], 
    theta_dot_deg_array[0], theta_dot_deg_array[-1],
)

py.legend(loc=0)
ode_plot.title_axis_labels()

py.savefig('pendulum_direction_field.svg')



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

